In [4]:
import backtrader as bt
import pandas as pd


In [11]:
df = pd.read_csv('AAPL.csv')
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
246,2022-06-27,142.699997,143.490005,140.970001,141.660004,141.660004,70207900
247,2022-06-28,142.130005,143.419998,137.320007,137.440002,137.440002,67083400
248,2022-06-29,137.460007,140.669998,136.669998,139.229996,139.229996,66242400
249,2022-06-30,137.250000,138.369995,133.770004,136.720001,136.720001,98964500
250,2022-07-01,136.039993,139.039993,135.660004,138.929993,138.929993,71007500


In [15]:
open_price = df['Open']#開巿價
open_price.head() 

0    140.070007
1    143.539993
2    141.580002
3    142.750000
4    146.210007
Name: Open, dtype: float64

In [ ]:
# 選擇方向?

In [16]:
# 計算 ADX
ADX_length = 15
ADX_thershold= 14

def calcADX(data, ADX_length):
    increase_level = 5 # data['High'][0] - data['High'][-1] 上升量
    decrease_level = 1 # data['Low'][-1] - data['Low'][0]  下降量
    if increase_level > decrease_level and increase_level > 0:
        DM_plus = increase_level
    else:
        DM_plus = 0 # 正DM
    if decrease_level > increase_level and decrease_level > 0:
        DM_minus = decrease_level
    else:
        DM_minus = 0 #負DM
    
    ATR = bt.indicators.ATR(data ,ADX_length) # 去拎datas[0]

    _plus = 100 * bt.indicators.SmoothedMovingAverage(DM_plus, ADX_length)/ATR # 如果係NAN就拎最近既數唔識寫
    _minus = 100 * bt.indicators.SmoothedMovingAverage(DM_minus, ADX_length)/ATR
    sum = _plus + _minus
    if sum == 0:
        _adx = 100 * bt.indicators.SmoothedMovingAverage((abs(_plus - _minus) /  1), ADX_length)
    else:
        _adx = 100 * bt.indicators.SmoothedMovingAverage((abs(_plus - _minus) /  sum), ADX_length)
    
    results = [_plus, _minus, _adx]
    return results




In [ ]:
# 計算 ADX
ADX_length = 15
ADX_thershold= 14

def calcADX(data, ADX_length):
    increase_level = 5 # data['High'][0] - data['High'][-1] 上升量
    decrease_level = 1 # data['Low'][-1] - data['Low'][0]  下降量
    if increase_level > decrease_level and increase_level > 0:
        DM_plus = increase_level
    else:
        DM_plus = 0 # 正DM
    if decrease_level > increase_level and decrease_level > 0:
        DM_minus = decrease_level
    else:
        DM_minus = 0 #負DM
    
    ATR = bt.indicators.ATR(data ,ADX_length) # 去拎datas[0]

    # 正DI
    DI_plus = 100 * bt.indicators.MovAv(DM_plus, ADX_length)/ATR # 如果係NAN就拎最近既數唔識寫
    # 負DI
    DI_minus = 100 * bt.indicators.MovAv(DM_minus, ADX_length)/ATR
    
    DI_sum = DI_plus + DI_minus
    if DI_sum == 0:
        DI_sum = 1
    # ADX = 100 × (+DI − -DI)絕對值的 (平滑移動平均) ÷ (+DI + -DI)
    _adx = 100 * (bt.indicators.SmoothedMovingAverage((abs(DI_plus - DI_minus)), ADX_length) /  sum)
    
    results = [DI_plus, DI_minus, _adx]
    return results

In [17]:
# ADX 在backtrader的計算

# 計算 ADX
ADX_length = 15
ADX_thershold= 14

ADX = bt.indicators.ADX
DI_plus = bt.indicators.PlusDirectionalIndicator (PlusDI)

In [ ]:
# 計算 Range Filter
sampling_period = 51
range_multiplier = 1.4


In [ ]:
# 計算 volume
volume_multiplier = 1.2
volume_length = 26
volume_multiplier_breakouts = 2.2
volume_length_breakouts = 26

# volume 超過 26日 平衡值1.2倍
Volume_condt = data[0]['Volume'] > bt.indicators.sma(datas['Volume'], volume_length)*volume_multiplier

# volume 超過 26日 平衡值1.2倍
Volume_Breakouts_condt = data[0]['Volume'] > bt.indicators.sma(datas['Volume'], volume_length)*volume_multiplier

In [40]:
volume_multiplier = 2.2
if (221 > 100*volume_multiplier):
    Volume_Breakouts_condt = True
else: 
    Volume_Breakouts_condt = False
Volume_Breakouts_condt

True

In [ ]:
# 計算MACD
MACD_fast_length = 9
MACD_slow_length = 13
signal_length = 19

In [ ]:
# 計算SAR 轉向指標
SAR_start = 0.5
SAR_int = 0.2 # 加速系數
SAR_max = 0.3 # Extreme Point 極限點位


In [41]:
import math
# 計算 MAMA FAMA
Cloud_length = 25

PI = 2 * math.asin(1) #3.141592653589793

def nz(data):
    if math.isnan(data):
        data = 0
    else:
        data = data
    return data

def hilbertTransform(src):
    result = 0.0962 * src + 0.5769 * nz(src[2]) - 0.5769 * nz(src[4]) - 0.0962 * nz(src[6])
    return result

def computeComponent(src, mesaPeriodMult):
    result = hilbertTransform(src) * mesaPeriodMult
    return result

def computeAlpha(src, fastLimit, slowLimit):
    mesaPeriod = 0.0
    mesaPeriodMult = 0.075 * nz(mesaPeriod[1]) + 0.54

    
    smooth = (4 * src + 3 * nz(src[1]) + 2 * nz(src[2]) + nz(src[3])) / 10

    detrender = computeComponent(smooth, mesaPeriodMult)
    
    # Compute InPhase and Quadrature components
    I1 = nz(detrender[3])
    Q1 = computeComponent(detrender, mesaPeriodMult)
    
    # Advance the phase of I1 and Q1 by 90 degrees
    jI = computeComponent(I1, mesaPeriodMult)
    jQ = computeComponent(Q1, mesaPeriodMult)
    
    I2 = 0.0
    Q2 = 0.0
    
    # Phasor addition for 3 bar averaging
    I2 = I1 - jQ
    Q2 = Q1 + jI
    
    # Smooth the I and Q components before applying the discriminator
    I2 = 0.2 * I2 + 0.8 * nz(I2[1])
    Q2 = 0.2 * Q2 + 0.8 * nz(Q2[1])
    
    # Homodyne Discriminator
    Re = I2 * nz(I2[1]) + Q2 * nz(Q2[1])
    Im = I2 * nz(Q2[1]) - Q2 * nz(I2[1])
    
    Re = 0.2 * Re + 0.8 * nz(Re[1])
    Im = 0.2 * Im + 0.8 * nz(Im[1])
    
    if Re != 0 and Im != 0:
        mesaPeriod = 2 * PI / math.atan(Im / Re)
    
    if mesaPeriod > 1.5 * nz(mesaPeriod[1]):
        mesaPeriod = 1.5 * nz(mesaPeriod[1])
    
    if mesaPeriod < 0.67 * nz(mesaPeriod[1]):
        mesaPeriod = 0.67 * nz(mesaPeriod[1])

    if mesaPeriod < 6:
        mesaPeriod = 6

    if mesaPeriod > 50:
        mesaPeriod = 50

    mesaPeriod = 0.2 * mesaPeriod + 0.8 * nz(mesaPeriod[1])

    phase = 0.0

    if I1 != 0:
        phase = (180 / PI) * math.atan(Q1 / I1)
    
    deltaPhase = nz(phase[1]) - phase
    
    if  deltaPhase < 1:
        deltaPhase = 1

    alpha = fastLimit / deltaPhase
    
    if  alpha < slowLimit:
        alpha = slowLimit

    return [alpha, alpha/2.0]

def change (datas, len): # 
    return datas[0] - datas[-len]
    
src = df['Open']
er = abs(change(src,len)) / sum(abs(change(src)),len) #src 係開巿價 
[a,b] = computeAlpha(src, er, er*0.1)

mama = 0.0
mama = a * src + (1 - a) * nz(mama[1]) #前一個mama

fama = 0.0
fama = b * mama + (1 - b) * nz(fama[1]) #前一個fama


alpha = pow((er * (b - a)) + a, 2)

kama = 0.0
kama = alpha * src + (1 - alpha) * nz(kama[1]) #前一個kama

if kama > kama[1]:
    cloud_Long = True
else:
    cloud_Long = False

if kama < kama[1]:
    cloud_Short = True
else:
    cloud_Short = False


3.141592653589793

In [45]:
import talib as ta
import numpy as np
a = np.array([1,1,4,3,5,8,13], dtype=float)
a[1:] - a[:-1]

ta.MOM(a, 2)








array([nan, nan,  3.,  2.,  1.,  5.,  8.])

In [ ]:
# 計算RSI
RSI_length = 24
RSI_source = df["Open"]
#df['lc']=df['close'].shift(1)

rsi = bt.indicators.rsi(RSI_source, RSI_length)

# 不要在起買區買入, 超買區買入不會有好價錢
if rsi < 70:
    rsi_long = True
else:
    rsi_long = False

# 不要在起賣區賣出, 超賣區賣出不會有好價錢
if rsi > 30:
    rsi_short = True
else: 
    rsi_short = False


In [ ]:
# 計算Support and Resistance
left = 6
right = 8



In [ ]:
# 計算BOLINGER BANDS
BB_timeframe = ""
BB_source = df["High"]

In [48]:
# Scalping所使用的參數
ACT_scalping = True # 開關, 未必用到
scalping_lenght = 12 # HiLoLen
EMA_length_fast = 10
EMA_length_middle = 120
EMA_length_slow = 500

filter_BW = False # 唔知係乜
pullback_lookback = 5
use_Heikin_Ashi_Candles = True # Heikin Ashi

In [ ]:
# 計算RMI
RMI_length = 26
RMI_momentum = 29
RMI_oversell = 31
RNI_overbought = 63



In [ ]:
# 計算ATR
pp_period = 12
ATR_factor = 7
ATR_Period = 2
